In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from foottraffic.awb_model.models import model, variablemodels, priormodel, dataloading
import numpy as np
import pandas as pd
from einops import rearrange, reduce, repeat


In [25]:
mff = pd.DataFrame()
mff["Period"] = repeat(pd.Series(pd.date_range("2020-04-01", freq="W-MON", periods=156)).to_numpy(), "d -> (d g p v)", g=10, p=5, v=20)
mff["Geography"] = repeat(
  np.array(["US", "UK", "DE", "FR", "JP",
   "CA", "AU", "BR", "MX", "KR"]), 
  "g -> (d g p v)", d=156, p=5, v=20)
mff["Product"] = repeat(
  np.array(["Watches", "Phones", "Laptops", "Tablets", "Desktops"]),
  "p -> (d g p v)", d=156, g=10, v=20)
mff['VariableName'] = repeat(
  np.array(["Sales", "Price", "Promotion", "Stock", "Competition"] + [f"media_var_{i}" for i in range(15)]),
  "v -> (d g p v)", d=156, g=10, p=5)
mff['VariableValue'] = np.random.normal(size=mff.shape[0])
mff["Outlet"] = "Total"
mff["Campaign"] = "Total"
mff["Creative"] = "Total"
mff.head()

,Period,Geography,Product,VariableName,VariableValue,Outlet,Campaign,Creative
0,2020-04-06,US,Watches,Sales,-1.178103,Total,Total,Total
1,2020-04-06,US,Watches,Price,0.123505,Total,Total,Total
2,2020-04-06,US,Watches,Promotion,-0.858016,Total,Total,Total
3,2020-04-06,US,Watches,Stock,-0.250310,Total,Total,Total
4,2020-04-06,US,Watches,Competition,-0.701515,Total,Total,Total


In [26]:
mff.to_csv("mff.csv", index=False)

In [27]:
my_mff = dataloading.MFF.load_from_file("mff.csv")
my_mff.set_index_rows(['Geography', 'Product'])
my_mff.to_bundle("example_dataset")

/Users/mreda/miniconda3/envs/foottraffic/lib/python3.11/site-packages/pydantic/main.py:352: UserWarning: Pydantic serializer warnings:
  Expected `tuple[literal['Geography','Period','Product','Outlet','Campaign','Creative'], ...]` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


In [30]:
my_mff.analytic_dataframe(indexed=True).head()

Competition_Total_Total_Total  \
Geography Product  Period                                      
AU        Desktops 2020-04-06                      -0.668955   
                   2020-04-13                      -0.958664   
                   2020-04-20                       0.577616   
                   2020-04-27                      -1.750890   
                   2020-05-04                       0.197452   

                               Price_Total_Total_Total  \
Geography Product  Period                                
AU        Desktops 2020-04-06                 0.174888   
                   2020-04-13                 0.054043   
                   2020-04-20                 0.203846   
                   2020-04-27                -1.369900   
                   2020-05-04                 0.367659   

                               Promotion_Total_Total_Total  \
Geography Product  Period                                    
AU        Desktops 2020-04-06                     1.041951   
                   2020-04-13                     0.726187   
                   2020-04-20                    -1.076217   
                   2020-04-27                    -0.646932   
                   2020-05-04                     0.212854   

                               Sales_Total_Total_Total  \
Geography Product  Period                                
AU        Desktops 2020-04-06                 0.051555   
                   2020-04-13                 1.153100   
                   2020-04-20                 0.791275   
                   2020-04-27                 0.174080   
                   2020-05-04                -0.680308   

                               Stock_Total_Total_Total  \
Geography Product  Period                                
AU        Desktops 2020-04-06                 0.065232   
                   2020-04-13                -2.145637   
                   2020-04-20                -0.416459   
                   2020-04-27                 0.756465   
                   2020-05-04                 1.619176   

                               media_var_0_Total_Total_Total  \
Geography Product  Period                                      
AU        Desktops 2020-04-06                       0.211632   
                   2020-04-13                      -1.452407   
                   2020-04-20                      -0.423842   
                   2020-04-27                      -0.250443   
                   2020-05-04                      -0.406937   

                               media_var_1_Total_Total_Total  \
Geography Product  Period                                      
AU        Desktops 2020-04-06                      -0.279096   
                   2020-04-13                      -0.105823   
                   2020-04-20                      -0.877859   
                   2020-04-27                      -0.087381   
                   2020-05-04                      -0.265654   

                               media_var_10_Total_Total_Total  \
Geography Product  Period                                       
AU        Desktops 2020-04-06                        0.656968   
                   2020-04-13                       -0.008716   
                   2020-04-20                        0.305064   
                   2020-04-27                        1.088403   
                   2020-05-04                        1.617760   

                               media_var_11_Total_Total_Total  \
Geography Product  Period                                       
AU        Desktops 2020-04-06                       -0.730644   
                   2020-04-13                        0.526612   
                   2020-04-20                        0.230916   
                   2020-04-27                       -0.801312   
                   2020-05-04                        0.075905   

                               media_var_12_Total_Total_Total  \
Geography Product  Period                          

In [28]:
model.FoottrafficModel.new_from_dataset("example_dataset")

/Users/mreda/miniconda3/envs/foottraffic/lib/python3.11/site-packages/pydantic/main.py:352: UserWarning: Pydantic serializer warnings:
  Expected `tuple[literal['Geography','Period','Product','Outlet','Campaign','Creative'], ...]` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


In [16]:
my_model = model.FoottrafficModel.load("first model")

In [17]:
my_model.data.analytic_dataframe().head()

,Geography,Product,Period,Media 1_Outlet_Campaign_Creative,Media 2_Outlet_Campaign_Creative
0,Canada,Jewelry,2021-12-06,0.761563,1.195581
1,Canada,Jewelry,2021-12-13,0.632769,0.884852
2,Canada,Jewelry,2021-12-20,-1.822554,0.779260
3,Canada,Jewelry,2021-12-27,0.033166,-0.642591
4,Canada,Jewelry,2022-01-03,-0.527984,-0.680240


In [18]:
my_model.data.analytic_dataframe(indexed=True).head()

Media 1_Outlet_Campaign_Creative  \
Geography Product Period                                         
Canada    Jewelry 2021-12-06                          0.761563   
                  2021-12-13                          0.632769   
                  2021-12-20                         -1.822554   
                  2021-12-27                          0.033166   
                  2022-01-03                         -0.527984   

                              Media 2_Outlet_Campaign_Creative  
Geography Product Period                                        
Canada    Jewelry 2021-12-06                          1.195581  
                  2021-12-13                          0.884852  
                  2021-12-20                          0.779260  
                  2021-12-27                         -0.642591  
                  2022-01-03                         -0.680240

In [19]:
my_model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [x]


d:\MattReda\miniforge\envs\foottraffic\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 33 seconds.


In [20]:
my_model.save('first model aug 2')

d:\MattReda\miniforge\envs\foottraffic\Lib\site-packages\pydantic\main.py:415: UserWarning: Pydantic serializer warnings:
  Expected `tuple[literal['Geography','Period','Product','Outlet','Campaign','Creative'], ...]` but got `list` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_json(


In [ ]:
my_

In [ ]:
my_model.predict()

In [ ]:
my_model.trace_plot()

In [ ]:
my_model.save_outputs()

In [ ]:
my_model.summary()